In [2]:
import os
import pandas as pd
import geopandas as gpd
from geo.Geoserver import Geoserver
import fiona

In [3]:
data_dir='/home/ajai-krishna/work/qgis_geoserver/India states data'
style_dir='/home/ajai-krishna/work/qgis_geoserver/Styles/'
style_files = [os.path.join(style_dir,i) for i in os.listdir(style_dir)]
data_files=[os.path.join(data_dir,i) for i in os.listdir(data_dir)]
data_files

['/home/ajai-krishna/work/qgis_geoserver/India states data/India_states_simple-daily-indices_ssp245_2021-2040.gpkg',
 '/home/ajai-krishna/work/qgis_geoserver/India states data/India_states_pr-mm-ond_ssp585_2021-2040.gpkg',
 '/home/ajai-krishna/work/qgis_geoserver/India states data/India_states_tw-year-max_ssp585_2021-2040.gpkg',
 '/home/ajai-krishna/work/qgis_geoserver/India states data/India_states_max-5day-precip_ssp585_2021-2040.gpkg',
 '/home/ajai-krishna/work/qgis_geoserver/India states data/India_states_OND-CDD_ssp245_2021-2040.gpkg',
 '/home/ajai-krishna/work/qgis_geoserver/India states data/India_states_MAM-CSD_ssp585_2021-2040.gpkg',
 '/home/ajai-krishna/work/qgis_geoserver/India states data/India_states_MAM-rh_ssp585_2021-2040.gpkg',
 '/home/ajai-krishna/work/qgis_geoserver/India states data/India_states_max-5day-precip_ssp245_2021-2040.gpkg',
 '/home/ajai-krishna/work/qgis_geoserver/India states data/India_states_hwdi_ssp245_2021-2040.gpkg',
 '/home/ajai-krishna/work/qgis_ge

In [4]:
style_files.pop(5)

'/home/ajai-krishna/work/qgis_geoserver/Styles/.ipynb_checkpoints'

In [5]:
style_files

['/home/ajai-krishna/work/qgis_geoserver/Styles/simple_style_pr-mm-ond_ssp585.sld',
 '/home/ajai-krishna/work/qgis_geoserver/Styles/simple_style_OND-max-1day-precip_ssp245.sld',
 '/home/ajai-krishna/work/qgis_geoserver/Styles/simple_style_tw-MAM-max_ssp245.sld',
 '/home/ajai-krishna/work/qgis_geoserver/Styles/simple_style_MAM-tw_ssp585.sld',
 '/home/ajai-krishna/work/qgis_geoserver/Styles/simple_style_pr-mm-ond_ssp245.sld',
 '/home/ajai-krishna/work/qgis_geoserver/Styles/simple_style_OND-Heavy-rainfall-days-20mm_ssp245.sld',
 '/home/ajai-krishna/work/qgis_geoserver/Styles/simple_style_OND-Heavy-rainfall-days-10mm_ssp245.sld',
 '/home/ajai-krishna/work/qgis_geoserver/Styles/simple_style_pr-mm-year_ssp585.sld',
 '/home/ajai-krishna/work/qgis_geoserver/Styles/simple_style_OND-SDII_ssp585.sld',
 '/home/ajai-krishna/work/qgis_geoserver/Styles/simple_style_OND-CDD_ssp245.sld',
 '/home/ajai-krishna/work/qgis_geoserver/Styles/simple_style_JJAS-SDII_ssp585.sld',
 '/home/ajai-krishna/work/qgis_g

In [ ]:
# datastore_name = os.path.splitext(os.path.basename(data_files))
#  for i in data_files]
# datastore_name

In [6]:
server= Geoserver('http://localhost:8080/geoserver', username='admin', password='geoserver')

stores={}
for i in data_files:
    datastore_name = os.path.splitext(os.path.basename(i))[0]
    print(f'created {datastore_name}')
    print(f'{i}')
    store = server.create_gpkg_datastore(path=i)
    stores[datastore_name] = store
    print(f"created data_store:{datastore_name}")

len(style_files),len(data_files)

In [14]:
def geoserver_upload(data_files,style_files,workspace='Climate_data_Indian_states'):
    stores=[]
    style=[]
    for data in data_files:        
        
        datastore_name = os.path.splitext(os.path.basename(data))[0]
        try:
            server.delete_featurestore(featurestore_name = datastore_name, workspace = workspace)
            print(f'deleted {datastore_name}')

        except:
            print(f'creating {datastore_name}')
        layers = fiona.listlayers(data)
        layer_name = layers[0] if layers else datastore_name
        server.create_gpkg_datastore(path=data)
        print(f'created {datastore_name}')
        print(f'{data}')
        stores.append(datastore_name)
        print(f'datastore created {datastore_name}')

    for styles in style_files:
        style_name = os.path.splitext(os.path.basename(styles))[0]
        server.delete_style(style_name=style_name, workspace=workspace)
        print(f'deleted {style_name}')
        server.upload_style(path = styles, name = style_name, workspace = workspace)
        style.append(style_name)
        print(f'style uploaded {style_name}')
    
        
    # for layer1,style1 in zip(stores,style):    
    #     print(f'publishing {style1} to layer {layer1}')
    server.publish_style(layer_name = layer_name, style_name = style_name, workspace=workspace)
    print(f'published {style1} to layer {layer_name}')


In [17]:
geoserver_upload(data_files,style_files,workspace='Climate_data_Indian_states')

deleted featurestore India_states_simple-daily-indices_ssp245_2021-2040
created datastore India_states_simple-daily-indices_ssp245_2021-2040


GeoserverException: Status : 500 - b"Resource named 'simple-daily-indices_ssp245_2021-2040' already exists in store: 'India_states_simple-daily-indices_ssp245_2021-2040'"

In [ ]:
# existing_styles = server.get_styles(workspace='Climate_data_Indian_states')
# existing_style_names = set(existing_styles)
styles={}
for i in style_files:
    
    style_name = os.path.splitext(os.path.basename(i))[0]
    
    server.delete_style(style_name=style_name, workspace='Climate_data_Indian_states')
    print(f'deleted style {i}')
    
    style=server.upload_style(path = i, name = style_name, workspace = 'Climate_data_Indian_states')
    
    styles[style_name]=style
    print(f'style uploaded {style_name}')

In [ ]:
print(f'{styles}\n{stores}')

In [ ]:
server.publish_style(layer_name = '/home/ajai-krishna/work/qgis_geoserver/India states data/India_states_CDD_ssp585_2021-2040.gpkg',style_name='/home/ajai-krishna/work/qgis_geoserver/Styles/simple_style_CDD_ssp585.sld',
                           workspace='Climate_data_Indian_states')

In [ ]:
server.create_coveragestyle(raster_path=r'path\to\raster\file.tiff', style_name='style_1', workspace='demo',
                         color_ramp='RdYiGn')

In [ ]:
server.create_gpkg_datastore(path = 'India states data/India_states_CDD_ssp245_2021-2040.gpkg')

In [ ]:
server.upload_style(path=r'/home/ajai-krishna/work/qgis_geoserver/Styles/simple_style_CDD_ssp585.sld', workspace='Climate_data_Indian_states')